Financial Data Analysis

IMPORTING IMPORTANT LIBRARIES

In [2]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import nltk
import textblob
import talib as ta

In [11]:
import sys
import os


sys.path.append(os.path.abspath("../scripts"))

import sys
import os


sys.path.append(os.path.abspath("../scripts"))
from utils import *
from language_processing import *

data_utils = DataUtils()
LangProcess = LanguageProcessing



EXPLORATORY DATA ANALYSIS

In [4]:
news = data_utils.load_news_data()

In [7]:
news.head()

,headline,url,publisher,stock
date,,,,
2020-06-05 14:30:54+00:00,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,A
2020-06-03 14:45:20+00:00,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,A
2020-05-26 08:30:07+00:00,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,A
2020-05-22 16:45:06+00:00,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,A
2020-05-22 15:38:59+00:00,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,A


In [8]:
news.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1407328 entries, 2020-06-05 14:30:54+00:00 to 2011-05-12 04:00:00+00:00
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   headline   1407328 non-null  object
 1   url        1407328 non-null  object
 2   publisher  1407328 non-null  object
 3   stock      1407328 non-null  object
dtypes: object(4)
memory usage: 53.7+ MB


In [9]:
news.shape

(1407328, 4)

In [10]:
news.isnull().sum()

headline     0
url          0
publisher    0
stock        0
dtype: int64

In [5]:
news[news.duplicated()]

,headline,url,publisher,stock
date,,,,
2015-07-07 04:00:00+00:00,API Tech Reports Q2 Loss $0.08 Vs Exp Loss $0....,https://www.benzinga.com/news/earnings/15/07/5...,Eddie Staley,ATNY
2015-07-01 04:00:00+00:00,"Top Performing Industries For July 1, 2015",https://www.benzinga.com/news/15/07/5642290/to...,Lisa Levin,GME


Observation

* The data doesn't contain null or duplicated values.

Descriptive Statistics

In [6]:
news["headline length"] = news["headline"].apply(lambda x: len(x.split()))

In [7]:
news.describe()

,headline length
count,1.407328e+06
mean,1.141671e+01
std,6.352997e+00
min,1.000000e+00
25%,7.000000e+00
50%,1.000000e+01
75%,1.300000e+01
max,7.700000e+01


In [21]:
news_publishers = (
    news.groupby(by=["publisher"])["headline"].count().sort_values(ascending=False)
)

In [22]:
px.line(
    news.resample("ME").size(),
    title="Publicaitons per month",
    labels={"date": "Months", "value": "Number of news published"},
)


In [23]:
px.line(
    news.groupby([news.index.year])["headline"].count(),
    labels={"value": "Number of news published", "date": "Year"},
)

In [26]:
px.line(
    news.groupby([news.index.day])["headline"].count(),
    labels={"value": "Number of news published", "date": "Dates"},
)

Text Analysis

Sentiment analysis